## Demonstration of the Item-Based Collaborative Recommender

This system leverages collaborative filtering by analyzing how user-item interactions bridges items
Therefore, it focuses on the user-item relation.

It recommends articles that these similar users have engaged with, aiming to provide personalized suggestions. The model's performance is evaluated using MAP@K and NDCG@K metrics.

In [6]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

import polars as pl
import numpy as np

from parquet_data_reader import ParquetDataReader
from utils.data_preprocessing import DataProcesser
from models.collaborative.item_based_CF import ItemBasedCollaborativeRecommender
parquet_reader = ParquetDataReader()

pl.Config.set_tbl_cols(-1)

polars.config.Config

In [7]:
train_behaviors_df = parquet_reader.read_data("../../data/train/behaviors.parquet")
train_history_df = parquet_reader.read_data("../../data/train/history.parquet")
articles_df = parquet_reader.read_data("../../data/articles.parquet")
document_vectors_df = parquet_reader.read_data("../../data/document_vector.parquet")

## Data import & Preprocessing

In [8]:
print("articles_df has the size:         ", articles_df.shape)
print("train_behaviors_df has the size:  ", train_behaviors_df.shape)
print("train_history_df has the size:    ", train_history_df.shape)
print("document_vectors_df has the size: ", document_vectors_df.shape)

articles_df has the size:          (20738, 21)
train_behaviors_df has the size:   (232887, 17)
train_history_df has the size:     (15143, 5)
document_vectors_df has the size:  (125541, 2)


### Validation set

In [9]:
test_behaviours_df = parquet_reader.read_data('../../data/validation/behaviors.parquet')
test_behaviours_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
96791,null,2023-05-28 04:21:24,9.0,null,2,"[9783865, 9784591, … 9784710]",[9784696],22548,false,null,null,null,false,142,72.0,100.0
96798,null,2023-05-28 04:31:48,46.0,null,2,"[9782884, 9783865, … 9784648]",[9784281],22548,false,null,null,null,false,143,16.0,28.0
96801,null,2023-05-28 04:30:17,14.0,null,2,"[9784648, 7184889, … 9781983]",[9784444],22548,false,null,null,null,false,143,12.0,24.0
96808,null,2023-05-28 04:27:19,22.0,null,2,"[9784607, 9695098, … 9781983]",[9781983],22548,false,null,null,null,false,142,125.0,80.0
96810,null,2023-05-28 04:29:47,23.0,null,2,"[9781983, 7184889, … 9781520]",[9784642],22548,false,null,null,null,false,142,null,null


In [10]:
# Combine train and test behaviors
combined_df = pl.concat([train_behaviors_df, test_behaviours_df])

# Generate a random mask for splitting
n = combined_df.height  # Total number of rows
test_mask = np.random.rand(n) < 0.30  # 30% test, 70% train

# Apply the mask
test_behaviors_df = combined_df.filter(test_mask)
train_behaviors_df = combined_df.filter(~test_mask)

# Verify the split
print(f"Train size: {train_behaviors_df.shape[0]}, Test size: {test_behaviors_df.shape[0]}")

Train size: 334245, Test size: 143289


### Table contents

The information on news articles. As we are going to perform user-user CF, this table is not neccesary

In [11]:
articles_df.head()

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,[3150850],"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative"""
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,[3006712],"""article_default""","""https://ekstrabladet.dk/underh…",[],[],"[""Underholdning"", ""Film og tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive"""
3012771,"""Morten Bruun fyret i Sønderjys…","""FODBOLD: Morten Bruun fyret me…",2023-06-29 06:20:39,false,"""Kemien mellem spillerne i Supe…",2006-05-01 14:28:40,[3177953],"""article_default""","""https://ekstrabladet.dk/sport/…",[],[],"[""Erhverv"", ""Kendt"", … ""Ansættelsesforhold""]",142,"[196, 199]","""sport""",null,null,null,0.8241,"""Negative"""
3023463,"""Luderne flytter på landet""","""I landets tyndest befolkede om…",2023-06-29 06:20:43,false,"""Det frække erhverv rykker på l…",2007-03-24 08:27:59,[3184029],"""article_default""","""https://ekstrabladet.dk/nyhede…",[],[],"[""Livsstil"", ""Erotik""]",118,[133],"""nyheder""",null,null,null,0.7053,"""Neutral"""
3032577,"""Cybersex: Hvornår er man utro?""","""En flirtende sms til den flott…",2023-06-29 06:20:46,false,"""De fleste af os mener, at et t…",2007-01-18 10:30:37,[3030463],"""article_default""","""https://ekstrabladet.dk/sex_og…",[],[],"[""Livsstil"", ""Partnerskab""]",565,[],"""sex_og_samliv""",null,null,null,0.9307,"""Neutral"""


Each file consists of seven days of impression logs. The train_behaviors_df table contains all information about interactions between users and items, and can be used as a basis for user-user CF. <strong>Therefore we only need this table</strong>.

In [12]:
train_behaviors_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
149474,null,2023-05-24 07:47:53,13.0,null,2,"[9778623, 9778682, … 9778728]",[9778657],139836,false,null,null,null,false,759,7.0,22.0
153068,9778682,2023-05-24 07:09:04,78.0,100.0,1,"[9778657, 9778669, … 9778682]",[9778669],151570,false,null,null,null,false,1976,45.0,100.0
153070,9777492,2023-05-24 07:13:14,26.0,100.0,1,"[9020783, 9778444, … 9778628]",[9778628],151570,false,null,null,null,false,1976,4.0,18.0
153071,9778623,2023-05-24 07:11:08,125.0,100.0,1,"[9777492, 9774568, … 9775990]",[9777492],151570,false,null,null,null,false,1976,26.0,100.0
153075,9777492,2023-05-24 07:13:58,26.0,100.0,1,"[9778500, 9776420, … 9020783]",[9777034],151570,false,null,null,null,false,1976,7.0,16.0


Each file consists of users' click histories collected over 21 days period. This table does contain the same values as the train_behaviours_df, but as that table is easier to work with we will use train_behaviours_df over this one

In [13]:
train_history_df.head()

user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
u32,list[datetime[μs]],list[f32],list[i32],list[f32]
13538,"[2023-04-27 10:17:43, 2023-04-27 10:18:01, … 2023-05-17 20:36:34]","[100.0, 35.0, … 100.0]","[9738663, 9738569, … 9769366]","[17.0, 12.0, … 16.0]"
14241,"[2023-04-27 09:40:18, 2023-04-27 09:40:33, … 2023-05-17 17:08:41]","[100.0, 46.0, … 100.0]","[9738557, 9738528, … 9767852]","[8.0, 9.0, … 12.0]"
20396,"[2023-04-27 12:30:44, 2023-04-27 12:31:34, … 2023-05-17 10:59:44]","[100.0, 59.0, … 13.0]","[9738760, 9738355, … 9769679]","[49.0, 34.0, … 4.0]"
34912,"[2023-04-29 07:12:49, 2023-04-29 13:01:18, … 2023-05-18 05:06:40]","[100.0, 35.0, … 27.0]","[9741802, 9741804, … 9770882]","[153.0, 7.0, … 5.0]"
37953,"[2023-04-27 19:17:10, 2023-04-27 19:17:27, … 2023-05-17 21:29:22]","[14.0, 28.0, … 18.0]","[9739205, 9739202, … 9769306]","[4.0, 16.0, … 6.0]"


List of vectors for each article. This is used to describe the items. It could be used for item-item CF, but is not relevant to user-user CF.  <strong>This table will therefore not be used</strong>

In [14]:
document_vectors_df.head()

article_id,document_vector
i32,list[f32]
3000022,"[0.065424, -0.047425, … 0.035706]"
3000063,"[0.028815, -0.000166, … 0.027167]"
3000613,"[0.037971, 0.033923, … 0.063961]"
3000700,"[0.046524, 0.002913, … 0.023423]"
3000840,"[0.014737, 0.024068, … 0.045991]"


From the analasys we see that we only need train_behaviour_df to perform user-user CF

## Preprocessing

### Remove non-needed values

We see that we have several items that are not required for performing user-user CF

In [15]:
train_behaviors_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
149474,null,2023-05-24 07:47:53,13.0,null,2,"[9778623, 9778682, … 9778728]",[9778657],139836,false,null,null,null,false,759,7.0,22.0
153068,9778682,2023-05-24 07:09:04,78.0,100.0,1,"[9778657, 9778669, … 9778682]",[9778669],151570,false,null,null,null,false,1976,45.0,100.0
153070,9777492,2023-05-24 07:13:14,26.0,100.0,1,"[9020783, 9778444, … 9778628]",[9778628],151570,false,null,null,null,false,1976,4.0,18.0
153071,9778623,2023-05-24 07:11:08,125.0,100.0,1,"[9777492, 9774568, … 9775990]",[9777492],151570,false,null,null,null,false,1976,26.0,100.0
153075,9777492,2023-05-24 07:13:58,26.0,100.0,1,"[9778500, 9776420, … 9020783]",[9777034],151570,false,null,null,null,false,1976,7.0,16.0


All information that does not describe a user, or a user-item interaction can therefore be removed

In [16]:
irelevant_columns = ["impression_time", "device_type", "article_ids_inview", "article_ids_clicked", "session_id", "next_read_time", "next_scroll_percentage"]
train_behaviors_df = train_behaviors_df.drop(irelevant_columns)
train_behaviors_df.head()

impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,i32,f32,f32,u32,bool,i8,i8,i8,bool
149474,null,13.0,null,139836,false,null,null,null,false
153068,9778682,78.0,100.0,151570,false,null,null,null,false
153070,9777492,26.0,100.0,151570,false,null,null,null,false
153071,9778623,125.0,100.0,151570,false,null,null,null,false
153075,9777492,26.0,100.0,151570,false,null,null,null,false


The remaining items are the ones that can be used. But already here we see that we have several features with lacking information. We should therefore treat this

### Account for missing values

We see here that alot of the behaviours contain missing values. The therefore have to either remove or replace the values

In [17]:
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(334245, 10)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,235263,0,236787,0,0,311366,327806,325426,0


In [18]:
train_behaviors_df = train_behaviors_df.filter(train_behaviors_df["article_id"].is_not_null())
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(98982, 10)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,2569,0,0,92183,96745,95573,0


We see that of 70421, are there between 65-68000 missing values for gender, postcode and age. We therefore remove these as there is no use subsidizing them

In [19]:
train_behaviors_df = train_behaviors_df.drop(["gender", "postcode", "age"])
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(98982, 7)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,is_subscriber
u32,u32,u32,u32,u32,u32,u32
0,0,0,2569,0,0,0


We still see that 2570/98967 rows are missing a scroll percentage. As this is very low (<3%) we can easily replace this. Intitially we just set scroll to 0

In [20]:
train_behaviors_df = train_behaviors_df.fill_null(strategy="zero")

### Account for multiple instances of the same article and user

By checking rows where the user_id and article_id are the same we see that we have 9855 instances where the user has read the same article multiple times

In [21]:
duplicates = train_behaviors_df.group_by(["article_id", "user_id"]).count().filter(pl.col("count") > 1)

print(duplicates)

shape: (12_336, 3)
┌────────────┬─────────┬───────┐
│ article_id ┆ user_id ┆ count │
│ ---        ┆ ---     ┆ ---   │
│ i32        ┆ u32     ┆ u32   │
╞════════════╪═════════╪═══════╡
│ 9784863    ┆ 1160889 ┆ 4     │
│ 9733858    ┆ 2164397 ┆ 2     │
│ 9783236    ┆ 92186   ┆ 2     │
│ 9757574    ┆ 793135  ┆ 4     │
│ 9774376    ┆ 1956681 ┆ 2     │
│ …          ┆ …       ┆ …     │
│ 9769580    ┆ 1076722 ┆ 2     │
│ 9781502    ┆ 2562535 ┆ 2     │
│ 9784856    ┆ 755721  ┆ 2     │
│ 9790344    ┆ 818258  ┆ 2     │
│ 9773295    ┆ 1006267 ┆ 2     │
└────────────┴─────────┴───────┘


C:\Users\magnu\AppData\Local\Temp\ipykernel_9084\4233841252.py:1: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  duplicates = train_behaviors_df.group_by(["article_id", "user_id"]).count().filter(pl.col("count") > 1)


We see that we need to combine these duplicate rows. We therefore propose that for multiple instances of the same article and user, we combine the readtime and select the largest scroll percentage. This way we can preserve the data without having duplicates

In [22]:
dataProcesser = DataProcesser()
behaviors_df = dataProcesser.collaborative_filtering_preprocess()
train_df, test_df = dataProcesser.random_split(behaviors_df, test_ratio=0.2)
print(train_df.head())

shape: (5, 4)
┌────────────┬─────────┬────────────────┬────────────┐
│ article_id ┆ user_id ┆ total_readtime ┆ max_scroll │
│ ---        ┆ ---     ┆ ---            ┆ ---        │
│ i32        ┆ u32     ┆ f32            ┆ f32        │
╞════════════╪═════════╪════════════════╪════════════╡
│ 9774652    ┆ 707124  ┆ 6.0            ┆ 100.0      │
│ 9785992    ┆ 2049682 ┆ 77.0           ┆ 100.0      │
│ 9779807    ┆ 1581880 ┆ 95.0           ┆ 100.0      │
│ 9783334    ┆ 566670  ┆ 18.0           ┆ 100.0      │
│ 9782438    ┆ 1243338 ┆ 17.0           ┆ 100.0      │
└────────────┴─────────┴────────────────┴────────────┘


## Model Fit

This first model uses readtime and read percentage interactions to compare the user interactions 

In [23]:
recommender = ItemBasedCollaborativeRecommender(train_df)

In [24]:
recommender = ItemBasedCollaborativeRecommender(train_df)
recommender.fit()

{9774652: [(9777075, np.float64(0.9700219328990135)),
  (9781624, np.float64(0.28708521488694894)),
  (9774840, np.float64(0.060067036204756286)),
  (9786280, np.float64(0.04748881235275493)),
  (9774120, np.float64(0.01538162584572944)),
  (9778422, np.float64(0.013564856212442744)),
  (9781870, np.float64(0.010975736452448204)),
  (9776337, np.float64(0.009643989996265034)),
  (9783990, np.float64(0.00961921066154836)),
  (9758882, np.float64(0.005110127669961995))],
 9785992: [(9734834, np.float64(1.0801962324613612e-08)),
  (9773307, np.float64(3.6122629332169254e-09)),
  (9787034, np.float64(3.003892024011634e-09)),
  (9774557, np.float64(2.5903603706467493e-09)),
  (9773292, np.float64(1.3551512134668542e-09)),
  (9765759, np.float64(6.713284372850126e-10)),
  (9777492, np.float64(6.713281042181052e-10)),
  (9788524, np.float64(6.713277711511978e-10)),
  (9775568, np.float64(6.713158917648343e-10)),
  (9776071, np.float64(6.372415928268538e-10))],
 9779807: [(9787855, np.float64(

This first model just compares all artilces read by users when comparing users

In [25]:
binary_recommender = ItemBasedCollaborativeRecommender(train_df, binary_model=True)
binary_recommender.fit()

{9774652: [(9790572, np.float64(0.10803395392536075)),
  (9773279, np.float64(0.10372487162750843)),
  (9773744, np.float64(0.10360994288720204)),
  (9553271, np.float64(0.09901475429766748)),
  (9775297, np.float64(0.09901475429766748)),
  (9718212, np.float64(0.09901475429766748)),
  (8487537, np.float64(0.09901475429766748)),
  (9771910, np.float64(0.09901475429766748)),
  (9578459, np.float64(0.09901475429766748)),
  (9621194, np.float64(0.09901475429766748))],
 9785992: [(9785986, np.float64(0.15294382258037453)),
  (9714168, np.float64(0.14433756729740643)),
  (9766011, np.float64(0.13608276348795434)),
  (9782237, np.float64(0.12286829679574762)),
  (9789997, np.float64(0.1203642301214124)),
  (9783824, np.float64(0.12009611535381537)),
  (8912725, np.float64(0.11785113019775795)),
  (9394753, np.float64(0.11785113019775795)),
  (9474656, np.float64(0.11785113019775795)),
  (9498042, np.float64(0.11785113019775795))],
 9779807: [(9777750, np.float64(0.11749469908920929)),
  (978

Of the original 15143 users, only 9194 can be accounted for with the current solution. This should be changed in the future

## Model presentation

### Article Recommendation

In [26]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9785596, 9777846, 9777969, 9790293, 9749857]
reccomended for user  620796 :  [9782115, 9784489, 9778257, 9790811, 9498042]
reccomended for user  1067393 :  []
reccomended for user  1726258 :  [9786209, 9777324, 9778842, 9771235, 9779520]
reccomended for user  17205 :  [9787510, 9789404, 8518755, 9776688, 9627627]


In [27]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", binary_recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9774864, 9673979, 9695259, 8494970, 9717962]
reccomended for user  620796 :  [9765326, 9670286, 9563007, 9306867, 9678305]
reccomended for user  1067393 :  []
reccomended for user  1726258 :  [9790917, 9670286, 9678305, 9440508, 9667501]
reccomended for user  17205 :  [9768638, 9705425, 9482380, 9627627, 9640315]


### Evaluation Scores

#### Without ability to reccomend read articles

The complex model only reccomending articles the user has not yet read

In [28]:
results = recommender.evaluate_recommender(test_df, k=10, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.004424778761061947),
 'NDCG@K': np.float64(0.010301293760244117)}

The binary reccomender model only reccomending articles the user has not yet read

In [29]:
results = binary_recommender.evaluate_recommender(test_df, k=10, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.003703703703703704),
 'NDCG@K': np.float64(0.018178661988416098)}

#### With ability to reccomend previously read articles

The complex model reccomending articles the user, even if they have read them before

In [30]:
results = recommender.evaluate_recommender(test_df, k=10, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.0048543689320388345),
 'NDCG@K': np.float64(0.009734881844453546)}

The binary recommender model reccomending articles the user, even if they have read them before

In [31]:
results = binary_recommender.evaluate_recommender(test_df, k=10, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.004807692307692308),
 'NDCG@K': np.float64(0.014135052442850916)}

## Model Experimentation

In [32]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=100, allow_read_articles=True)
results = set(test_behaviours_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9786243, 9787524, 9781902, 9784591, 9783824, 9786111, 9776916, 9779615, 9788705, 9789473, 9428643, 9783334, 9782315, 9756075, 9787441, 9782722, 9786821, 9782726, 9786566, 9789896, 9787465, 9788362, 9791049, 9782092, 9780815, None, 9783509, 9772508, 9786718, 9786719, 9787487, 9790942, 9783655, 9786351, 9780849, 9781875, 9788661, 9781878, 9787510, 9786618, 9673979, 9780348, 9781887}
[9785596, 9777846, 9777969, 9790293, 9749857, 9775985, 9774764, 9772710, 9780193, 9759717, 9759345, 9787261, 4265340, 9776917, 9772221, 9778155, 9771355, 9778326, 9779724, 9698332, 9785062, 9764444, 9786210, 9766140, 9785973, 9777200, 9760091, 9771065, 9777397, 9789427, 9785475, 9776147, 9492305, 9735085, 9769193, 9772557, 9777464, 9759544, 9714168, 9781389, 9778686, 9625415, 9782722, 9781316, 9779204, 9773873, 9783276, 9777767, 9772032, 9776553, 9772923, 9780697, 9789494, 9777319, 9769348, 9778219, 7086478, 9656349, 9784869, 9789810, 9789664, 9777026, 9080070, 9722202, 9539706, 9765759, 9777492, 9788524, 97

In [33]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=100, allow_read_articles=True)
results = set(test_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9771916, 9781902, 9784591, 9774352, 9776916, 9775518, 9769504, 9769531, 9774142, 9672256, 9782722, 9772355, 9774532, 9787465, 9788362, 9776855, 9772508, 9790942, 9738729, 9776497, 9787510, 9786618, 9778939}
[9785596, 9777846, 9777969, 9790293, 9749857, 9775985, 9774764, 9772710, 9780193, 9759717, 9759345, 9787261, 4265340, 9776917, 9772221, 9778155, 9771355, 9778326, 9779724, 9698332, 9785062, 9764444, 9786210, 9766140, 9785973, 9777200, 9760091, 9771065, 9777397, 9789427, 9785475, 9776147, 9492305, 9735085, 9769193, 9772557, 9777464, 9759544, 9714168, 9781389, 9778686, 9625415, 9782722, 9781316, 9779204, 9773873, 9783276, 9777767, 9772032, 9776553, 9772923, 9780697, 9789494, 9777319, 9769348, 9778219, 7086478, 9656349, 9784869, 9789810, 9789664, 9777026, 9080070, 9722202, 9539706, 9765759, 9777492, 9788524, 9775568, 9776691, 9767233, 9778448, 9654458, 9771113, 9488208, 9658650, 9776570, 9774864, 9783751, 9778021, 9775573, 9769679, 9660886, 9769367, 9385951, 9756546, 9578459, 9514605,

#### Binary Item Based Recommender Metrics


In [34]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=100, allow_read_articles=True)
results = set(test_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9771916, 9781902, 9784591, 9774352, 9776916, 9775518, 9769504, 9769531, 9774142, 9672256, 9782722, 9772355, 9774532, 9787465, 9788362, 9776855, 9772508, 9790942, 9738729, 9776497, 9787510, 9786618, 9778939}
[9785596, 9777846, 9777969, 9790293, 9749857, 9775985, 9774764, 9772710, 9780193, 9759717, 9759345, 9787261, 4265340, 9776917, 9772221, 9778155, 9771355, 9778326, 9779724, 9698332, 9785062, 9764444, 9786210, 9766140, 9785973, 9777200, 9760091, 9771065, 9777397, 9789427, 9785475, 9776147, 9492305, 9735085, 9769193, 9772557, 9777464, 9759544, 9714168, 9781389, 9778686, 9625415, 9782722, 9781316, 9779204, 9773873, 9783276, 9777767, 9772032, 9776553, 9772923, 9780697, 9789494, 9777319, 9769348, 9778219, 7086478, 9656349, 9784869, 9789810, 9789664, 9777026, 9080070, 9722202, 9539706, 9765759, 9777492, 9788524, 9775568, 9776691, 9767233, 9778448, 9654458, 9771113, 9488208, 9658650, 9776570, 9774864, 9783751, 9778021, 9775573, 9769679, 9660886, 9769367, 9385951, 9756546, 9578459, 9514605,

In [35]:
from utils.evaluation import perform_model_evaluation
metrics = perform_model_evaluation(binary_recommender, test_data=test_df, k=5)
metrics

{'precision@k': np.float64(0.004076396807297606),
 'recall@k': np.float64(0.011042909955429569),
 'fpr@k': np.float64(0.0021917271209687178)}

In [36]:
from utils.evaluation import append_model_metrics
append_model_metrics(metrics, "item based binary")

#### Complex Item Based Recommender Metrics

In [37]:
metrics_complex = perform_model_evaluation(recommender, test_data=test_df, k=5)
print(metrics_complex)

append_model_metrics(metrics_complex, "item based complex")

{'precision@k': np.float64(0.002166476624857469), 'recall@k': np.float64(0.00252611823621784), 'fpr@k': np.float64(0.0021964402696801203)}


### Diversity Evaluation
Calculates the aggrigate diversity of the recommender model recommendations, and appends the result to the `/evaluation_summary/model_overview_diversity.csv`-file. 

In [38]:
users_df = train_history_df["user_id"].unique()
users_df

user_id
u32
10068
10200
10201
10623
10701
…
2590015
2590054
2590471


In [39]:
articles_ids_df = articles_df["article_id"].unique()
articles_ids_df

article_id
i32
3001353
3003065
3012771
3023463
3032577
…
9803492
9803505
9803525


In [40]:
from utils.evaluation import aggregate_diversity
from utils.evaluation import append_aggregate_diversity

print(recommender.item_similarity_matrix)

diversity = aggregate_diversity(recommender, articles_df, users_df=users_df, user_sample=1000)

print("Diversity")
print(diversity)

append_aggregate_diversity(diversity, "item based CF")

{9774652: [(9777075, np.float64(0.9700219328990135)), (9781624, np.float64(0.28708521488694894)), (9774840, np.float64(0.060067036204756286)), (9786280, np.float64(0.04748881235275493)), (9774120, np.float64(0.01538162584572944)), (9778422, np.float64(0.013564856212442744)), (9781870, np.float64(0.010975736452448204)), (9776337, np.float64(0.009643989996265034)), (9783990, np.float64(0.00961921066154836)), (9758882, np.float64(0.005110127669961995))], 9785992: [(9734834, np.float64(1.0801962324613612e-08)), (9773307, np.float64(3.6122629332169254e-09)), (9787034, np.float64(3.003892024011634e-09)), (9774557, np.float64(2.5903603706467493e-09)), (9773292, np.float64(1.3551512134668542e-09)), (9765759, np.float64(6.713284372850126e-10)), (9777492, np.float64(6.713281042181052e-10)), (9788524, np.float64(6.713277711511978e-10)), (9775568, np.float64(6.713158917648343e-10)), (9776071, np.float64(6.372415928268538e-10))], 9779807: [(9787855, np.float64(0.3819135886017413)), (9780329, np.flo

### Gini coefficient 
Calculates the Gini coefficient for the recommender model and appends it to the `/output/model_overview_gini.csv` file.

In [41]:
from utils.evaluation import gini_coefficient
from utils.evaluation import append_gini_coefficient

gini = gini_coefficient(recommender, users_df, articles_ids_df=articles_ids_df, user_sample=1000)
append_gini_coefficient(gini, "item based CF")

Sampling users
Computing Gini coefficient
[9781932, 9781987, 9788411, 9775793, 9772957, 9775800, 9778728, 9778661, 9773488, 9482380, 9773356, 9780302, 9787332, 9788677, 9688780, 9773376, 9773295, 9775785, 9784947, 9788462, 9784879, 9778943, 9787767, 9766007, 9773203, 9784064, 9787243, 9783159, 9788095, 9779577, 9778813, 5712427, 9762122, 9779071, 9777636, 9774972, 9766592, 9784947, 9775183, 9779538, 9788947, 9777750, 9776176, 9786563, 9761926, 9776710, 9784808, 9783643, 9790822, 9787230, 9776710, 9775855, 9783643, 9784808, 9790822, 9786204, 9774032, 9788921, 9779092, 9791165, 9726237, 9770051, 9773392, 9783790, 9783990, 9775752, 9786293, 9787469, 9531745, 9781932, 9784138, 9784160, 9785174, 9782057, 9786955, 9627627, 8518755, 9659139, 9705425, 9776191, 9782115, 9789743, 9775003, 9777529, 9780498, 9673564, 9749014, 9787855, 9670069, 9630091, 9785596, 9775985, 9774764, 9777846, 9777969, 9724617, 9779489, 9789974, 9777406, 9772284, 9768687, 9776710, 9784808, 9783643, 9790822, 9774944, 977

### Carbon Footprint
This section creates an emissions.csv file in the "output"-folder
It utilizes the code_carbon (`codecarbon EmissionsTracker`) to record the carbon footprint of the `fit` and the `recommend` methods of the model.

In [42]:
from utils.evaluation import track_model_energy

print("\nCarbon footprint of the recommender:")
footprint = track_model_energy(recommender, "item_based", user_id=test_user_id, n=5)
footprint

[codecarbon INFO @ 13:28:04] [setup] RAM Tracking...
[codecarbon INFO @ 13:28:04] [setup] CPU Tracking...
[codecarbon WARNING @ 13:28:04] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU




Carbon footprint of the recommender:


[codecarbon WARNING @ 13:28:06] We saw that you have a 13th Gen Intel(R) Core(TM) i7-13700H but we don't know it. Please contact us.
[codecarbon INFO @ 13:28:06] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 13:28:06] [setup] GPU Tracking...
[codecarbon INFO @ 13:28:06] No GPU found.
[codecarbon INFO @ 13:28:06] >>> Tracker's metadata:
[codecarbon INFO @ 13:28:06]   Platform system: Windows-10-10.0.26100-SP0
[codecarbon INFO @ 13:28:06]   Python version: 3.11.9
[codecarbon INFO @ 13:28:06]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 13:28:06]   Available RAM : 15.731 GB
[codecarbon INFO @ 13:28:06]   CPU count: 20
[codecarbon INFO @ 13:28:06]   CPU model: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 13:28:06]   GPU count: None
[codecarbon INFO @ 13:28:06]   GPU model: None
[codecarbon INFO @ 13:28:10] Saving emissions data to file c:\Users\magnu\NewDesk\An.sys\TDT4215\recommender_system\demostrations\output\item_based_fit_

{'fit': ({9774652: [(9777075, np.float64(0.9700219328990135)),
    (9781624, np.float64(0.28708521488694894)),
    (9774840, np.float64(0.060067036204756286)),
    (9786280, np.float64(0.04748881235275493)),
    (9774120, np.float64(0.01538162584572944)),
    (9778422, np.float64(0.013564856212442744)),
    (9781870, np.float64(0.010975736452448204)),
    (9776337, np.float64(0.009643989996265034)),
    (9783990, np.float64(0.00961921066154836)),
    (9758882, np.float64(0.005110127669961995))],
   9785992: [(9734834, np.float64(1.0801962324613612e-08)),
    (9773307, np.float64(3.6122629332169254e-09)),
    (9787034, np.float64(3.003892024011634e-09)),
    (9774557, np.float64(2.5903603706467493e-09)),
    (9773292, np.float64(1.3551512134668542e-09)),
    (9765759, np.float64(6.713284372850126e-10)),
    (9777492, np.float64(6.713281042181052e-10)),
    (9788524, np.float64(6.713277711511978e-10)),
    (9775568, np.float64(6.713158917648343e-10)),
    (9776071, np.float64(6.372415928